In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
from time import sleep
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time 
from unidecode import unidecode
import json 
import requests
from selenium.webdriver.common.proxy import Proxy, ProxyType

def getProxy():
    keydepip = '26191c8fd4436643e7e3488c5c0bb0e8'
    authen_ip = "59.153.238.246"
    url = f'http://api.fasthideproxy.com/socks5?apikey={keydepip}&ip={authen_ip}&changenow=1'
    print(url)
    response = requests.get(url, timeout=50).json()
    proxies = {}
    print(response)
    ip = ""
    if response['code'] == 1:
        ip = response['message']

    elif response['code'] == 0:
        doi = response['message'].split(' ')[1]
        
    return ip

def initDriver():
    proxy_data = getProxy()
    prox = Proxy()
    prox.proxy_type = ProxyType.MANUAL
    prox.socks_version = 5
    prox.socks_proxy = proxy_data
    
    WINDOW_SIZE = "1000,2000"
    chrome_options = Options()
    chrome_options.proxy = prox
    #chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('disable-infobars')
    chrome_options.add_argument('--disable-gpu') if os.name == 'nt' else None  # Windows workaround
    chrome_options.add_argument("--verbose")
    chrome_options.add_argument("--no-default-browser-check")
    chrome_options.add_argument("--ignore-ssl-errors")
    chrome_options.add_argument("--allow-running-insecure-content")
    chrome_options.add_argument("--disable-web-security")
    chrome_options.add_argument("--disable-feature=IsolateOrigins,site-per-process")
    chrome_options.add_argument("--no-first-run")
    chrome_options.add_argument("--disable-notifications")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-translate")
    chrome_options.add_argument("--ignore-certificate-error-spki-list")
    chrome_options.add_argument("--ignore-certificate-errors")
    chrome_options.add_argument("--disable-blink-features=AutomationControllered")
    chrome_options.add_experimental_option('useAutomationExtension', False)
    prefs = {"profile.default_content_setting_values.notifications": 2}
    chrome_options.add_experimental_option("prefs", prefs)
    chrome_options.add_argument("--start-maximized")  # open Browser in maximized mode
    chrome_options.add_argument("--disable-dev-shm-usage")  # overcome limited resource problems
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
    chrome_options.add_argument('disable-infobars')

    driver = webdriver.Chrome(options=chrome_options)
    return driver



driver = initDriver()

from selenium.webdriver.common.keys import Keys


url1 = "https://www.tripadvisor.com.vn/Hotels-g293925"
url2 = "-Ho_Chi_Minh_City-Hotels.html#BODYCON"
domain = 'https://www.tripadvisor.com.vn'





http://api.fasthideproxy.com/socks5?apikey=26191c8fd4436643e7e3488c5c0bb0e8&ip=59.153.238.246&changenow=1
{'code': 1, 'message': '103.170.123.148:53297'}


In [ ]:
driver.find_element(By.XPATH,"//button[@class='rmyCe _G B- z _S c Wc wSSLS pexOo sOtnj']").click()

In [11]:
target_urls = [(url1 + str(i) + url2) for i in list(map(lambda x: 30 * x, range(1, 15)))]
target_url = "https://www.tripadvisor.com.vn/Hotels-g293925-a_sort.POPULARITY-Ho_Chi_Minh_City-Hotels.html"
target_urls.insert(1, target_url)

driver.get(target_url)

data = []
for i in range(10):
    print(f"Lan {i} ----------------------------------")
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')
    listall = soup.findAll('div', {'class','jsTLT K'})
    for item in listall:
        data.append(item.find('a')['href'])
    
    driver.find_element(By.XPATH,"//a[@data-smoke-attr='pagination-next-arrow']").click()
    sleep(3)
    



Lan 0 ----------------------------------
Lan 1 ----------------------------------
Lan 2 ----------------------------------
Lan 3 ----------------------------------
Lan 4 ----------------------------------
Lan 5 ----------------------------------
Lan 6 ----------------------------------
Lan 7 ----------------------------------
Lan 8 ----------------------------------
Lan 9 ----------------------------------


In [29]:
with open("data.txt", "w") as f:
    for item in data:
        f.write(f"{item}\n")

### test item

In [3]:
domain = 'https://www.tripadvisor.com.vn'
link = "/Hotel_Review-g293925-d1952374-Reviews-A_EM_Saigon_Hotel-Ho_Chi_Minh_City.html"
target_url = domain+ link
driver.get(target_url)



### lấy đánh giá

In [1]:
def Get_Review(link):
    domain = 'https://www.tripadvisor.com.vn'
    target_url = domain + link
    driver.get(target_url)
    sleep(2)
    review = []
    for page in range(5):
        try:
            html_source = driver.page_source
            soup = BeautifulSoup(html_source, 'html.parser')
            listall = soup.findAll('div', {'class','azLzJ MI Gi z Z BB kYVoW'})
            for item in listall:
                rate = item.find('svg', {'class','UctUV d H0'}).get("aria-label")
                rv_title = item.find('span', {'class','JbGkU Cj'}).text
                rv_content = item.find('div', {'class','fIrGe _T'}).text
                rv_user = item.find('div', {'class','ScwkD _Z o S4 H3 Ci'}).text
                temp  ={
                        "rating" : rate,
                        "name" : rv_user,
                        "title" : rv_title,
                        "content" : rv_content
                    }
                review.append(temp)
                print(temp)
                
                
            driver.find_element(By.XPATH,"//a[@data-smoke-attr='pagination-next-arrow']").click()
            sleep(2)
        except Exception as e:
            print(f"error {link}", e)
            
    return review



In [6]:
data  = open("data.txt", "r").read().split("\n")
result = []
for link in data:
    temp = Get_Review(link)
    item_result = {
        "link" : link,
        "review" : temp
    }
    result.append(item_result)

error /Hotel_Review-g293925-d25442757-Reviews-La_Siesta_Premium_Sai_Gon-Ho_Chi_Minh_City.html Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[@data-smoke-attr='pagination-next-arrow']"}
  (Session info: chrome=120.0.6099.224); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7580E2142+3514994]
	(No symbol) [0x00007FF757D00CE2]
	(No symbol) [0x00007FF757BA76AA]
	(No symbol) [0x00007FF757BF1860]
	(No symbol) [0x00007FF757BF197C]
	(No symbol) [0x00007FF757C34EE7]
	(No symbol) [0x00007FF757C1602F]
	(No symbol) [0x00007FF757C328F6]
	(No symbol) [0x00007FF757C15D93]
	(No symbol) [0x00007FF757BE4BDC]
	(No symbol) [0x00007FF757BE5C64]
	GetHandleVerifier [0x00007FF75810E16B+3695259]
	GetHandleVerifier [0x00007FF758166737+4057191]
	GetHandleVerifier [0x00007FF75815E4E3+4023827]
	GetHandleVerifier [0x00007FF757E304F9+689705]

KeyboardInterrupt: 

In [ ]:
with open("review.json", "w", encoding="utf-8") as f:
    import json
    json.dump(result, f, ensure_ascii=False, indent=4)